In [1]:
import numpy as np
from numpy import linalg as LA

from matplotlib import pyplot as plt
from matplotlib import image

import os

from skimage import color 
from skimage.transform import rescale
from skimage.transform import resize

from sklearn.model_selection import train_test_split

import tkinter as tk
from tkinter import filedialog
from PIL import ImageTk, Image
from sklearn.metrics import mean_squared_error

In [2]:
image_datasets = r"FaceData"
file_names = os.listdir(image_datasets)

faces = []
ages = []
genders = []

for file_name in file_names:
    file_path = os.path.join(image_datasets,file_name)
    img = image.imread(file_path)
    
    ages.append(os.path.basename(file_name).split("_")[0])
    genders.append(os.path.basename(file_name).split("_")[1])
    
    gray_image = color.rgb2gray(img)
    gray_image_rescaled = rescale(gray_image,0.25,anti_aliasing=False)
    
    faces.append(gray_image_rescaled.flatten()) 

In [5]:
faces2np = np.array(faces).astype(float)
genders2np = np.array(genders).astype(int)*2 -1
ages2np = np.array(ages).astype(float)/100

In [6]:
X_train, X_test, y_genders_train, y_genders_test, y_ages_train, y_ages_test = train_test_split(faces2np,genders2np,ages2np, test_size=0.1, random_state=205)

In [31]:
def bias_matrix(dataset):
    biases = np.ones((dataset.shape[0],1))
    dataset = np.hstack((dataset,biases))
    return dataset

In [32]:
def one_dimensional_matrix_reshape(dataset):
    dataset_reshape = dataset.reshape((dataset.shape[0],1))
    return dataset_reshape

In [33]:
def linear_regression(X_train,y_train):
    X_train = bias_matrix(X_train)
    W = (LA.inv(X_train.T @ X_train)) @ X_train.T @ y_train
    return W

In [10]:
def Tikhonov_regularization(X_train,y_train,L): 
    biases = np.ones((X_train.shape[0],1))
    X_train = np.hstack((X_train,biases))
    W = (LA.inv((X_train.T @ X_train) + L* np.eye(X_train.shape[1])) @ X_train.T @ y_train)
    return W

In [30]:
def upload_pic():
    new_data = []
    file_path = filedialog.askopenfilename()
    
    if file_path:
        original_image = Image.open(file_path)
        rgb_image = original_image.convert('RGB')
        
        img = np.array(rgb_image)
            
        gray_new_image = color.rgb2gray(img)
        
        gray_new_image_rescaled = resize(gray_new_image, (50, 50), anti_aliasing=False)

        new_data.append(gray_new_image_rescaled.flatten()) 
        new_face = np.array(new_data).astype(float)
        
        return new_face,file_path

In [12]:
def display_image(image_path):
    
    img = Image.open(image_path)
    img = img.resize((300, 300))
    img_tk = ImageTk.PhotoImage(img)
    image_label.config(image=img_tk)
    image_label.image = img_tk

In [13]:
def sigmoid(y):
    return 1/(1+np.exp(-y))

def predict(X, W_hidden,W_output):

    y_pred = []
    for i in range(X.shape[0]):
        x_i = X[i:i+1]
        x_i_1 = np.hstack([x_i,[[1]]])
        x_h_i = sigmoid(W_hidden@x_i_1.T)
        x_h_i_1 = np.vstack([x_h_i,[1]])
        x_o = W_output@x_h_i_1
        y_pred.append([x_o[0][0]])

    return np.array(y_pred),x_h_i_1,x_i_1

In [12]:
W_ages_linear_regression = linear_regression(X_train,y_ages_train)
W_genders_linear_regression = linear_regression(X_train,y_genders_train)
np.save("W_ages_linear_regression.npy",W_ages_linear_regression)
np.save("W_genders_linear_regression.npy",W_genders_linear_regression)

In [13]:
W_ages_TR = Tikhonov_regularization(X_train,y_ages_train,L=10)
W_genders_TR = Tikhonov_regularization(X_train,y_genders_train,L=100)
np.save("W_ages_TR.npy",W_ages_TR)
np.save("W_genders_TR.npy",W_genders_TR)

In [14]:
k = 50
alpha_init = 0.00005
neuron_in_hidden_layer = 16

ann_X_train2 = X_train.copy()
ann_y_age_train = y_ages_train.copy()
ann_y_genders_train = y_genders_train.copy()




ann_X_test2 = X_test.copy()
ann_y_age_test = y_ages_test.copy()
ann_y_genders_test = y_genders_test.copy()




W_hidden = np.random.randn(neuron_in_hidden_layer,ann_X_train2.shape[1]+1)/k

W_output = np.random.randn(1,neuron_in_hidden_layer+1)/neuron_in_hidden_layer

small_value = 1e-8


A_hidden = np.zeros_like(W_hidden)
F_hidden = np.zeros_like(W_hidden)

A_output = np.zeros_like(W_output)
F_output = np.zeros_like(W_output)

rho = 0.999
rho_f = 0.9


for e in range(84):# for each epoch
    for i in range(X_train.shape[0]):
        #define input and output
        x_i = ann_X_train2[i:i+1]
        y_i = ann_y_age_train[i]

        y_pred_i,x_h_i_1,x_i_1 = predict(x_i, W_hidden,W_output)

        delta_k = y_i - y_pred_i

        #Gradient for output layer
        
        Gradient_output  = -delta_k*x_h_i_1.T

        A_output = (rho)*A_output + (1-rho)*(Gradient_output)**2

        F_output = rho_f*F_output + (1-rho_f)*(Gradient_output)

        alpha_t = alpha_init*(np.sqrt(1-rho**(i+1)))/(1-rho_f**(i+1))

        W_output = W_output - alpha_t*F_output/np.sqrt(A_output+small_value)


        delta_h = x_h_i_1*(1-x_h_i_1)*W_output.T*delta_k

        # Update each hidden unit
        Gradient_hidden = np.zeros_like(W_hidden)
        for h in range(W_hidden.shape[0]):

            #Gradient for hidden layer
            Gradient_hidden[h,:] =  - delta_h[h] * x_i_1

        # Gradient Descent rule

        A_hidden = (rho)*A_hidden + (1-rho)*(Gradient_hidden)**2

        F_hidden = rho_f*F_hidden + (1-rho_f)*(Gradient_hidden)

        alpha_t = alpha_init*(np.sqrt(1-rho**(i+1)))/(1-rho_f**(i+1))

        W_hidden = W_hidden - alpha_t*F_hidden/np.sqrt(A_hidden+small_value)


    y_pred,_,_ = predict(ann_X_train2, W_hidden,W_output)


np.save("Adam_age_hidden.npy",W_hidden)
np.save("Adam_age_output.npy",W_output)

In [15]:
k = 50
alpha_init = 0.00005
neuron_in_hidden_layer = 16

ann_X_train4 = X_train.copy()
ann_y_genders_train = y_genders_train.copy()



ann_X_test4 = X_test.copy()
ann_y_genders_test = y_genders_test.copy()

W_hidden = np.random.randn(neuron_in_hidden_layer,ann_X_train4.shape[1]+1)/k

W_output = np.random.randn(1,neuron_in_hidden_layer+1)/neuron_in_hidden_layer

small_value = 1e-8


A_hidden = np.zeros_like(W_hidden)
F_hidden = np.zeros_like(W_hidden)

A_output = np.zeros_like(W_output)
F_output = np.zeros_like(W_output)

rho = 0.999
rho_f = 0.9



for e in range(98):# for each epoch
    for i in range(X_train.shape[0]):
        #define input and output
        x_i = ann_X_train4[i:i+1]
        y_i = ann_y_genders_train[i]

        y_pred_i,x_h_i_1,x_i_1 = predict(x_i, W_hidden,W_output)

        delta_k = y_i - y_pred_i

        #Gradient for output layer
        Gradient_output  = -delta_k*x_h_i_1.T

        A_output = (rho)*A_output + (1-rho)*(Gradient_output)**2

        F_output = rho_f*F_output + (1-rho_f)*(Gradient_output)

        alpha_t = alpha_init*(np.sqrt(1-rho**(i+1)))/(1-rho_f**(i+1))

        W_output = W_output - alpha_t*F_output/np.sqrt(A_output+small_value)



        delta_h = x_h_i_1*(1-x_h_i_1)*W_output.T*delta_k

        # Update each hidden unit
        Gradient_hidden = np.zeros_like(W_hidden)
        for h in range(W_hidden.shape[0]):

            #Gradient for hidden layer
            Gradient_hidden[h,:] =  - delta_h[h] * x_i_1

        # Gradient Descent rule

        A_hidden = (rho)*A_hidden + (1-rho)*(Gradient_hidden)**2

        F_hidden = rho_f*F_hidden + (1-rho_f)*(Gradient_hidden)

        alpha_t = alpha_init*(np.sqrt(1-rho**(i+1)))/(1-rho_f**(i+1))

        W_hidden = W_hidden - alpha_t*F_hidden/np.sqrt(A_hidden+small_value)



    y_pred,_,_ = predict(ann_X_train4, W_hidden,W_output)


np.save("Adam_gender_hidden.npy",W_hidden)
np.save("Adam_gender_output.npy",W_output)

In [26]:
def LR_model():
    W_age = np.load("W_ages_linear_regression.npy")
    W_gender = np.load("W_genders_linear_regression.npy")
    
    new_data,img = upload_pic()
    new_testing_dataset = np.vstack((X_test.copy(),new_data))
    
    y_predict_age = bias_matrix(new_testing_dataset) @ (one_dimensional_matrix_reshape(W_age))
    y_predict_gender = bias_matrix(new_testing_dataset) @ (one_dimensional_matrix_reshape(W_gender))
    
    image_age = np.round(y_predict_age[-1] * 100,0)
    image_gender = "Male" if y_predict_gender[-1] < 0 else "Female"
    
    display_image(img)
    result_label.config(text=f"Age: {image_age}\n Gender: {image_gender}")


In [27]:

def TR_model():
    W_age = np.load("W_ages_TR.npy")
    W_gender = np.load("W_genders_TR.npy")
    
    new_data,img = upload_pic()
    new_testing_dataset = np.vstack((X_test.copy(),new_data))
    
    y_predict_age = bias_matrix(new_testing_dataset) @ (one_dimensional_matrix_reshape(W_age))
    y_predict_gender = bias_matrix(new_testing_dataset) @ (one_dimensional_matrix_reshape(W_gender))
    
    image_age = np.round(y_predict_age[-1] * 100,0)
    image_gender = "Male" if y_predict_gender[-1] < 0 else "Female"
    
    display_image(img)
    result_label.config(text=f"Age: {image_age}\n Gender: {image_gender}")


In [39]:

def ANN_model():
    W_age_hidden = np.load("Adam_age_hidden.npy")
    W_age_output = np.load("Adam_age_output.npy")
    
    W_gender_hidden = np.load("Adam_gender_hidden.npy")
    W_gender_output = np.load("Adam_gender_output.npy")

    new_data,img = upload_pic()
    new_testing_dataset = np.vstack((X_test.copy(),new_data))

    y_predict_age, _, _ = predict(new_testing_dataset, W_age_hidden, W_age_output)
    y_predict_gender, _, _ = predict(new_testing_dataset,W_gender_hidden, W_gender_output)


    image_age = np.round(y_predict_age[-1] * 100,0)
    image_gender = "Male" if y_predict_gender[-1][0] < 0 else "Female"
    print(y_predict_gender.shape)
    display_image(img)
    result_label.config(text=f"Age: {image_age}\n Gender: {image_gender}")
    


In [25]:
ANN_model()

0.554962490067375
0.7932446264073695
[[ 0.45575707]
 [-0.9221591 ]
 [-1.6724258 ]
 [-1.16881247]
 [ 0.90624687]
 [ 0.0290773 ]
 [-1.41419073]
 [-0.82230525]
 [ 0.04377624]
 [ 0.1567444 ]
 [ 0.39093936]
 [ 0.26025726]
 [-0.32184455]
 [-0.65115464]
 [-0.75549362]
 [-0.42868523]
 [ 0.06824228]
 [-0.81252267]
 [-0.3356553 ]
 [ 1.11850101]] [ 1  1  1  1  1 -1  1  1  1  1  1  1 -1  1  1  1  1  1  1 -1]


In [41]:
TK = tk.Tk()
TK.geometry("1000x800")
tk_button1 = tk.Button(TK, text="Linear Regression Model", width=30, font=('Arial', 15, 'italic'), command=lambda: print(LR_model()))
tk_button1.pack(padx=50, pady=30)

tk_button2 = tk.Button(TK, text="Tikhonov Regularization Model", width=30, font=('Arial', 15, 'italic'), command=lambda: print(TR_model()))
tk_button2.pack(padx=50, pady=30)

tk_button3 = tk.Button(TK, text="ANN Model", width=30, font=('Arial', 15, 'italic'), command=lambda: print(ANN_model()))
tk_button3.pack(padx=50, pady=30)

image_label = tk.Label(TK)
image_label.pack(padx=50, pady=30)

result_label = tk.Label(TK, text="", font=('Arial', 20))
result_label.pack(padx=50, pady=30)
TK.mainloop()

0.5551508252703472
0.7932446264073695
[[ 0.45575707]
 [-0.9221591 ]
 [-1.6724258 ]
 [-1.16881247]
 [ 0.90624687]
 [ 0.0290773 ]
 [-1.41419073]
 [-0.82230525]
 [ 0.04377624]
 [ 0.1567444 ]
 [ 0.39093936]
 [ 0.26025726]
 [-0.32184455]
 [-0.65115464]
 [-0.75549362]
 [-0.42868523]
 [ 0.06824228]
 [-0.81252267]
 [-0.3356553 ]
 [ 1.11850101]] [ 1  1  1  1  1 -1  1  1  1  1  1  1 -1  1  1  1  1  1  1 -1]
(978, 1)
None
0.5299641307928705
0.7932446264073695
[[ 0.45575707]
 [-0.9221591 ]
 [-1.6724258 ]
 [-1.16881247]
 [ 0.90624687]
 [ 0.0290773 ]
 [-1.41419073]
 [-0.82230525]
 [ 0.04377624]
 [ 0.1567444 ]
 [ 0.39093936]
 [ 0.26025726]
 [-0.32184455]
 [-0.65115464]
 [-0.75549362]
 [-0.42868523]
 [ 0.06824228]
 [-0.81252267]
 [-0.3356553 ]
 [ 1.11850101]] [ 1  1  1  1  1 -1  1  1  1  1  1  1 -1  1  1  1  1  1  1 -1]
(978, 1)
None
0.4448491747296528
0.7932446264073695
[[ 0.45575707]
 [-0.9221591 ]
 [-1.6724258 ]
 [-1.16881247]
 [ 0.90624687]
 [ 0.0290773 ]
 [-1.41419073]
 [-0.82230525]
 [ 0.0437762